In [35]:
import pandas as pd
import json
import re
import os

In [ ]:
event = pd.read_csv('Events.csv',encoding='utf-16')
event.drop(columns=['Severity','Subject','Text','Event Text','Area','Category','Importance','Class','Kind'], inplace=True)
event.fillna('', inplace=True)
tag_names = json.load(open('Tag_Name.json', 'r', encoding='utf-8')) #Only needed tags are loaded
pattern = '|'.join(map(re.escape, tag_names['tags']))
event = event[event['Tag Description'].str.contains(pattern, na=False)]
event = event[event['Priority'] != 0]
event.reset_index(drop=True, inplace=True)
event.head()

,Tag Name,Tag Description,Priority,In Time,Out Time
0,310A_LI_2302,"Tank 310A-TK-01, Level",3,2025-07-14 21:48:29.010,2025-07-15 05:09:13.210
1,310A_LI_2301,"Pump 310A-VP-01D/S, Level",1,2025-07-14 18:14:59.270,2025-07-15 01:47:50.010
2,310A_LI_2301,"Pump 310A-VP-01D/S, Level",1,2025-07-14 18:12:03.020,2025-07-14 18:14:59.270
3,310A_LI_2301,"Pump 310A-VP-01D/S, Level",3,2025-07-14 18:10:19.040,2025-07-14 18:12:03.020
4,310A_LI_2301,"Pump 310A-VP-01D/S, Level",1,2025-07-14 18:01:03.050,2025-07-14 18:06:21.080


In [ ]:
captors = pd.read_csv('Captors.csv',encoding='utf-16')
captors.drop(index=[0,1,2,3],axis=1,inplace=True)
captors.head()

,Time,310A_FI_4303,310A_DI_3302,310A_FQI_4303_SOLIDS,310A_PI_0316,310A_PI_0325,310A_PI_0578,310A_PI_0580,310A_FI_4301,310ASP01DI01SPM,310ASP01SI01SPM,310A_TI_5303_D,310A_TI_5303_E,310A_TI_5303_G,310A_TI_5304_D,310A_TI_5304_E,310A_TI_5304_F
4,06/18/2025 11:43:10,447,1.027,4.0,0.09,3.05,0.34,1.70,114,48,64,40,43,42,66,64,63
5,06/18/2025 13:43:10,215,1.027,4.0,0.10,2.77,0.46,2.69,0,48,59,41,43,42,73,71,70
6,06/18/2025 15:43:10,344,1.029,4.3,0.17,3.18,0.57,3.65,7,49,63,42,43,42,77,75,74
7,06/18/2025 17:43:10,298,1.073,9.3,0.98,5.02,0.63,3.51,0,59,56,72,56,62,70,69,68
8,06/18/2025 19:43:10,141,1.108,14.0,1.08,5.59,0.50,3.07,1,59,55,79,58,71,55,55,53


In [ ]:
def is_faulty(time, intervals):
    return any((time >= start) & (time <= end) for start, end in intervals)
event['In Time'] = pd.to_datetime(event['In Time'], format='%Y-%m-%d %H:%M:%S.%f')
event['Out Time'] = pd.to_datetime(event['Out Time'], format='%Y-%m-%d %H:%M:%S.%f')
captors['Time'] = pd.to_datetime(captors['Time'], format='%m/%d/%Y %H:%M:%S')
equipements = ['SP','VP','TK']
for equip in equipements:
    tmp = event[event['Tag Description'].str.contains(equip, na=False)]
    intervals = list(zip(tmp['In Time'], tmp['Out Time']))
    captors[f'faulty_{equip}'] = captors['Time'].apply(lambda t: 1 if is_faulty(t, intervals) else 0)